In [1]:
import numpy as np
from plotly import plotly
from plotly.grid_objs import Grid, Column
from plotly import graph_objs as go

Optimization Methods for Machine Learning

In [2]:
def f(x, y): return 0.4 * x ** 2 + x * y + 1.3 * y ** 2

def gradf(f , *xv, epsilon=1e-6):
    """
    Approximate the numerical gradient of f
    at around xv
    
    Parameters
    ----------
    f: function
        vectorized and derivable function
    xv: the evaluating points
    
    Returns
    -------
    np.array of shape (len(xv),);
    """
    nx = len(xv)
    xv_stack = np.array([xv for _ in range(nx)]).reshape(-1, nx)
    xv_stack_pos = (xv_stack + np.eye(nx) * epsilon).T
    xv_stack_neg = (xv_stack - np.eye(nx) * epsilon).T
    grad = (f(*xv_stack_pos) - f(*xv_stack_neg)) / (2 * epsilon)
    return grad

In [3]:
step = 0.5
xmin, xmax = -4, 4
ymin, ymax = -4, 4
xvals = np.arange(xmin, xmax, step)
yvals = np.arange(ymin, ymax, step)
X, Y = np.mgrid[xmin:xmax:step, ymin:ymax:step]
Z = f(X, Y)
Z.shape

(16, 16)

In [4]:
Y[(-2.50 <= X) * (X <= -2.40)]

array([-4. , -3.5, -3. , -2.5, -2. , -1.5, -1. , -0.5,  0. ,  0.5,  1. ,
        1.5,  2. ,  2.5,  3. ,  3.5])

In [5]:
Z[(-2.50 <= X) * (X <= -2.40)]

array([33.3  , 27.175, 21.7  , 16.875, 12.7  ,  9.175,  6.3  ,  4.075,
        2.5  ,  1.575,  1.3  ,  1.675,  2.7  ,  4.375,  6.7  ,  9.675])

In [6]:
x0 = np.array([-3, -3])
alpha = 0.1
learning_history = [x0]
learning_vals = [f(*x0)]
f1, f0 = f(*x0), 1e6
while abs(f1 / f0 - 1) > 1e-2:
    x0 = x0 - alpha * gradf(f, *x0)
    learning_history.append(x0)
    f0, f1 = f1, f(*x0)
    learning_vals.append(f1)

learning_history = np.r_[learning_history]

In [7]:
learning_history[1]

array([-2.46, -1.92])

In [8]:
learning_vals[1]

11.936159998938415

In [9]:
x, y = learning_history.T
nvals = 50
learning_xcols = [Column(x[:i+1], f"x{i}") for i, _ in enumerate(x[:nvals])]
learning_ycols = [Column(y[:i+1], f"y{i}") for i, _ in enumerate(y[:nvals])]
learning_zcols = [Column(learning_vals[:i+1], f"z{i}") for i, _ in enumerate(learning_vals[:nvals])]

In [10]:
columns_mesh = [Column(xvals, "xvals"), Column(yvals, "yvals"), Column(Z, "zvals")]
columns_learn = [*learning_xcols, *learning_ycols, *learning_zcols, *columns_mesh]

In [12]:
grid_learn = Grid(columns_learn)
plotly.grid_ops.upload(grid_learn, "learn01_grid", auto_open=False)

'https://plot.ly/~ger94/183/'

In [13]:
data = [
    {"data": [
    go.Scatter3d(xsrc=grid_learn.get_column_reference(f"x{0}"),
                 ysrc=grid_learn.get_column_reference(f"y{0}"),
                 zsrc=grid_learn.get_column_reference(f"z{0}"),
                 marker=dict(color='orange', size=5)),
    ]},
    go.Surface(xsrc=grid_learn.get_column_reference("xvals"),
               ysrc=grid_learn.get_column_reference("yvals"),
               zsrc=grid_learn.get_column_reference("zvals"),
               colorscale="Viridis",
               name="frame",
               opacity=0.8),
]

frames = [
    {"data": [
    go.Scatter3d(xsrc=grid_learn.get_column_reference(f"x{i}"),
                 ysrc=grid_learn.get_column_reference(f"y{i}"),
                 zsrc=grid_learn.get_column_reference(f"z{i}"),
                 marker=dict(color='orange', size=5)),
    go.Surface(xsrc=grid_learn.get_column_reference("xvals"),
               ysrc=grid_learn.get_column_reference("yvals"),
               zsrc=grid_learn.get_column_reference("zvals"),
               colorscale="Viridis",
               name="frame",
               opacity=0.8),
    ]}
for i, _ in enumerate(x[:nvals])]

layout = {
    "xaxis": {"range": [-4, 4], "autorange": False},
    "yaxis": {"range": [-4, 4], "autorange": False},
    "updatemenus": [{
        "buttons": [{
            "label": "Accion",
            "method": "animate",
            "args": [None,
                    {"frame": {"duration": 150, "redraw": True},
                     "transition": {"duration": 0}}]
        }],
        "pad": {"r":10, "t":87},
        "showactive": False,
        "type": "buttons"
    }]
}

figure = {
    "data": data,
    "layout": layout,
    "frames": frames
}

plotly.create_animations(figure, filename="learn01")

'https://plot.ly/~ger94/184/'

In [16]:
x0 = np.array([-3, -3])
alpha = 0.1
learning_history = [x0]
f1, f0 = f(*x0), 1e6
while abs(f1 / f0 - 1) > 1e-2:
    x0 = x0 - alpha * gradf(f, *x0)
    learning_history.append(x0)
    f0, f1 = f1, f(*x0)


learning_history = np.r_[learning_history]

In [20]:
from plotly.offline import iplot

In [21]:
xyrang = np.arange(-4, 4, step)
fig = go.FigureWidget(data=[
    go.Contour(z=Z, x=xyrang, y=xyrang, colorscale="Viridis", ncontours=30),
    go.Scatter(x=learning_history[:,1], y=learning_history[:,0], mode="lines+markers"),
])

iplot(fig)